In [434]:
import pandas as pd
import requests as httpRequest
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import date

In [4]:
Investing_BaseUrl = "https://in.investing.com/"

In [539]:
# Get the html content for the Investing.com Nifty 50 Stock List URL to extract the metadata for each stock

Investing_StockList = httpRequest.get("https://in.investing.com/indices/s-p-cnx-nifty-components").content
sel_Investing = Selector(text = Investing_StockList)

In [540]:
header = {
        'User-Agent':
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

In [541]:
Investing_StockList = httpRequest.get("https://in.investing.com/indices/s-p-cnx-nifty-components",headers=header)

In [542]:
soup = BeautifulSoup(Investing_StockList.text, 'html.parser')

In [552]:
count = 0
InvestingstockSeedList_Names = []
InvestingstockSeedList_Links = []

for news_list in soup.findAll('a',{'class':'js-instrument-page-link'}):
    if(count >= 50):
        continue
    count = count + 1
    InvestingstockSeedList_Names.append(news_list.get('title'))
    InvestingstockSeedList_Links.append(news_list.get('href'))
InvestingstockSeedList_Names

['AXIS Bank',
 'Adani Ports & SEZ',
 'Apollo Hospitals',
 'Asian Paints',
 'Bajaj Auto',
 'Bajaj Finance',
 'Bajaj Finserv',
 'Bharat Petroleum',
 'Bharti Airtel',
 'Britannia Industries',
 'Cipla',
 'Coal India',
 'Divi',
 'Dr. Reddy’s Labs',
 'Eicher Motors',
 'Grasim Industries',
 'HCL Tech',
 'HDFC Bank',
 'HDFC Life',
 'Hero MotoCorp',
 'Hindalco Industries',
 'Hindustan Unilever',
 'Housing Development Finance',
 'ICICI Bank',
 'ITC',
 'IndusInd Bank',
 'Infosys',
 'JSW Steel',
 'Kotak Mahindra Bank',
 'Larsen & Toubro',
 'Mahindra & Mahindra',
 'Maruti Suzuki',
 'NTPC',
 'Nestle India',
 'Oil & Natural Gas',
 'Power Grid',
 'Reliance Industries',
 'SBI',
 'SBI Life Insurance',
 'Shree Cements',
 'Sun Pharma',
 'Tata Consultancy',
 'Tata Consumer Products',
 'Tata Motors',
 'Tata Steel Ltd',
 'Tech Mahindra',
 'Titan Company',
 'UPL',
 'UltraTech Cement',
 'Wipro']

In [ ]:
#InvestingstockSeedList_Names = []
#InvestingstockSeedList_Links = []
#for tbody in soup.findAll('tbody'):
 #   for tr in tbody.findAll('tr',{'class':'common-table-item u-clickable'}):
       # for news_list in tr.find('a',{'class':'js-instrument-page-link'}):
        #    print(news_list.title)
  #      InvestingstockSeedList_Names.append(tr.a.get('title'))
   #     InvestingstockSeedList_Links.append(tr.a.get('href'))
#InvestingstockSeedList_Names

In [826]:
Stock_Code = ['AXISBANK' ,'ADANIPORTS','APOLLOHOSP','ASIANPAINT','BAJAJ-AUTO','BAJFINANCE','BAJAJFINSV','BPCL','BHARTIARTL',
              'BRITANNIA','CIPLA','COALINDIA','DIVISLAB','DRREDDY','EICHERMOT','GRASIM','HCLTECH','HDFCBANK','HDFCLIFE',
              'HEROMOTOCO','HINDALCO','HINDUNILVR',' HDFC ','ICICIBANK','ITC','INDUSINDBK','INFY','JSWSTEEL','KOTAKBANK',
              'LT','M&M','MARUTI','NTPC','NESTLEIND','ONGC','POWERGRID','RELIANCE','SBIN','SBILIFE','SHREECEM','SUNPHARMA',
              'TCS','TATACONSUM','TATAMOTORS','TATASTEEL','TECHM','TITAN','UPL','ULTRACEMCO','WIPRO']

In [827]:
StockMetadata = pd.DataFrame({"Stock_FullName": InvestingstockSeedList_Names,"Stock_Code": Stock_Code})

In [828]:
# Create a dataframe with columns to capture all the Stock Metadata
df_InvestingStockListMetadata = pd.DataFrame({"Stock_FullName": [],"InvestingUrl": [] ,"Stock_Code": []})

In [829]:
for (stockName, stockLink) in zip(InvestingstockSeedList_Names, InvestingstockSeedList_Links):
    df_InvestingStockListMetadata = df_InvestingStockListMetadata.append({
        'Stock_FullName' : str(stockName),
        'InvestingUrl'   : str(stockLink)
        },  ignore_index = True)

In [830]:
df_InvestingStockListMetadata = pd.merge(df_InvestingStockListMetadata,StockMetadata[['Stock_FullName','Stock_Code']],on='Stock_FullName',how='left')
df_InvestingStockListMetadata = df_InvestingStockListMetadata.drop('Stock_Code_x',axis=1)
df_InvestingStockListMetadata.rename(columns = {'Stock_Code_y':'Stock_Code'}, inplace = True)
df_InvestingStockListMetadata.loc[13,'Stock_Code'] = "DRREDDY"
for ind in df_InvestingStockListMetadata.index:
    df_InvestingStockListMetadata.loc[ind,'Stock_Code'] = df_InvestingStockListMetadata.loc[ind,'Stock_Code'].strip()

In [831]:
#Get the Nifty 50 stocks meta data and the stock news url
df_InvestingStockListMetadata["InvestingStockNews"] = ""
for eachStockIndex in df_InvestingStockListMetadata.index :
    df_InvestingStockListMetadata["InvestingStockNews"] = Investing_BaseUrl + df_InvestingStockListMetadata['InvestingUrl'] + "-news"
df_InvestingStockListMetadata.to_csv("C:/Users/Rupali/ISB/Foundational Project 1/InvestingStockListdata.csv")

In [812]:
# Create a dataframe with columns to capture all the Stock News with the dates
df_InvestingDateWiseStockNews = pd.DataFrame({"Stock_FullName": [], "Stock_Code": [], "Stock_News" : [],"Date" : []})

In [813]:
#Iterating over all the Nifty 50 stocks to get all the news for an individual stock based on the dates

row = 0
for stockIndex in df_InvestingStockListMetadata.index :
    stocknews_url = df_InvestingStockListMetadata.loc[stockIndex,"InvestingStockNews"]
    page = httpRequest.get(stocknews_url,headers=header)
    soup = BeautifulSoup(page.text, 'html.parser')
    for j in soup.select('div.content'):
        h3 = j.find('h3')
        df_InvestingDateWiseStockNews.loc[row,'Stock_FullName'] = df_InvestingStockListMetadata.loc[stockIndex,"Stock_FullName"]
        df_InvestingDateWiseStockNews.loc[row,'Stock_Code'] = df_InvestingStockListMetadata.loc[stockIndex,"Stock_Code"]
        df_InvestingDateWiseStockNews.loc[row,'Stock_News'] = h3.text.strip()
        for i in j.findAll('time'):
                if i.has_attr('datetime'):
                    df_InvestingDateWiseStockNews.loc[row,'Date'] = i.text
        row = row + 1

In [750]:
df_InvestingDateWiseStockNews.to_csv("C:/Users/Rupali/ISB/Foundational Project 1/InvestingStockNewsdata.csv", index = True)

In [816]:
df_InvestingDateWiseStockNews

,Stock_FullName,Stock_Code,Stock_News,Date
0,AXIS Bank,AXISBANK,Benchmark indices up post below estimate US in...,17 hours ago
1,AXIS Bank,AXISBANK,ICICI Bank is covered by 51 analysts on the St...,23 hours ago
2,AXIS Bank,AXISBANK,Benchmark indices fall marginally after rising...,"Aug 04, 2022 13:45"
3,AXIS Bank,AXISBANK,"Stocks in View: ITC, Zomato, Adani Group, Volt...","Aug 03, 2022 04:40"
4,AXIS Bank,AXISBANK,"Stocks Under Focus on Jul 25: Vodafone Idea, G...","Jul 25, 2022 04:01"
...,...,...,...,...
2061,Wipro,WIPRO,Pre-Open Cues: SGX Nifty Futures Signals Flat ...,NaN
2062,Wipro,WIPRO,Breaking: Dabur Chairman Amit Burman Quits; Me...,NaN
2063,Wipro,WIPRO,"Stocks Under Focus: Adani Enterprises, LIC, ON...",NaN
2064,Wipro,WIPRO,"Q1 Snippets of Key Stocks: Apollo Hospitals, B...",NaN


In [752]:
#Doing sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

In [753]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [799]:
#Function for generating the polarity of a single document 

def vader_sent_func(doc):
    sent_list = doc
    vader_doc = []
    vs_sent = analyzer.polarity_scores(sent_list)
    vader_doc.append(vs_sent)

    # Get the output as a DataFrame    
    doc_df = pd.DataFrame(vader_doc)
    return(doc_df)

In [800]:
#Sentiment Analysis for the whole corpus

def vader_corpus_func(doc):

    # Initializing the dataframe to add the output of single document
    vader_doc_df = pd.DataFrame(columns=['doc_index', 'neg', 'neu', 'pos', 'compound']) 
    
    for i in range(len(doc)):
        vader_doc = vader_sent_func(doc.loc[i])
        vader_doc.insert(0,'doc_index', i)
        vader_doc_df = pd.concat([vader_doc_df, vader_doc], axis=0)
    return(vader_doc_df) 

Sentiment_Investing_News = vader_corpus_func(df_InvestingDateWiseStockNews['Stock_News'])

In [801]:
SentimentScore_Investing_News = df_InvestingDateWiseStockNews[['Stock_Code','Date']].copy()
SentimentScore_Investing_News['Sentiment Score'] = Sentiment_Investing_News['compound'].to_numpy()

In [802]:
SentimentScore_Investing_News.dropna(axis=0,how = 'any',inplace=True)
SentimentScore_Investing_News

,Stock_Code,Date,Sentiment Score
0,AXISBANK,47 minutes ago,0.5106
1,AXISBANK,"Aug 04, 2022 13:45",0.2263
2,AXISBANK,"Aug 03, 2022 04:40",0.0000
3,AXISBANK,"Jul 25, 2022 04:01",0.0000
4,AXISBANK,"Jul 24, 2022 10:44",0.0000
...,...,...,...
2102,WIPRO,"Jul 20, 2022 21:50",0.0000
2103,WIPRO,"Jul 20, 2022 21:18",0.0000
2104,WIPRO,"Jul 20, 2022 20:37",0.4019
2105,WIPRO,"Jul 20, 2022 20:37",0.4404


In [803]:
for ind in SentimentScore_Investing_News.index:
    if(((SentimentScore_Investing_News['Date'][ind]).find('hours ago') != -1) or ((SentimentScore_Investing_News['Date'][ind]).find('minutes ago') != -1)):
        SentimentScore_Investing_News['Date'][ind] =  date.today()
SentimentScore_Investing_News['Date'] = pd.to_datetime(SentimentScore_Investing_News['Date']).dt.date

C:\Users\Rupali\AppData\Local\Temp/ipykernel_85296/4027556182.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SentimentScore_Investing_News['Date'][ind] =  date.today()


In [804]:
SentimentScore_Investing_News.to_csv("C:/Users/Rupali/ISB/Foundational Project 1/Sentiment_Investing_News.csv", index = True)

In [794]:
SentimentAvg_Investing_News = SentimentScore_Investing_News.groupby(['Stock_Code','Date']).agg({'Sentiment Score':['mean']})
SentimentAvg_Investing_News.columns = ['Mean_Sentiment_Score']
SentimentAvg_Investing_News = SentimentAvg_Investing_News.reset_index()

In [795]:
SentimentAvg_Investing_News.to_csv("C:/Users/Rupali/ISB/Foundational Project 1/SentimentAvg_Investing_News.csv", index = True)

In [796]:
SentimentAvg_Investing_News

,Stock_Code,Date,Mean_Sentiment_Score
0,ADANIPORTS,2022-08-03,0.104909
1,ADANIPORTS,2022-08-04,0.244169
2,ADANIPORTS,2022-08-05,0.133967
3,ADANIPORTS,2022-08-06,0.000000
4,ADANIPORTS,2022-08-08,0.000000
...,...,...,...
1138,WIPRO,2022-08-02,0.458800
1139,WIPRO,2022-08-03,0.000000
1140,WIPRO,2022-08-04,0.075433
1141,WIPRO,2022-08-05,0.000000


In [737]:
stockIndex = 0
# Prediction Mode
# Create a dataframe with columns to capture the Stock News for previous day
df_InvestingPredStock = pd.DataFrame({"Stock_FullName": [], "Stock_Code": [], "Stock_News" : [],"Date" : []})

def PredMode(stock_code,date):
    for ind in df_InvestingStockListMetadata.index:
         if(df_InvestingStockListMetadata.loc[ind,'Stock_Code'] == stock_code): 
                    stockIndex = ind
                    break
                    
    stocknews_url = df_InvestingStockListMetadata.loc[stockIndex,"InvestingStockNews"]

    page = httpRequest.get(stocknews_url,headers=header)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    row = 0
    for j in soup.select('div.content'):
        h3 = j.find('h3')     

        for i in j.findAll('time'):
                if i.has_attr('datetime'):
                    news_date = i.text
                    news_date = news_date[0:12]
                    if(news_date == date):
                        df_InvestingPredStock.loc[row,'Stock_FullName'] = df_InvestingStockListMetadata.loc[stockIndex,"Stock_FullName"]
                        df_InvestingPredStock.loc[row,'Stock_Code'] = df_InvestingStockListMetadata.loc[stockIndex,"Stock_Code"]
                        df_InvestingPredStock.loc[row,'Date'] = news_date
                        df_InvestingPredStock.loc[row,'Stock_News'] = h3.text.strip()
        row = row + 1
    return(df_InvestingPredStock)

In [738]:
df_PredStock = PredMode('ADANIPORTS',"Aug 09, 2022")
df_PredStock

,Stock_FullName,Stock_Code,Stock_News,Date
0,Adani Ports & SEZ,ADANIPORTS,"Adani Ports & SEZ earnings miss, Revenue beats...","Aug 09, 2022"


In [740]:
SentimentScore = vader_sent_func(df_PredStock)
PredStock_Sentiment = df_PredStock[['Stock_Code','Date']].copy()
PredStock_Sentiment['Mean_Sentiment_Score'] = SentimentScore['compound']
PredStock_Sentiment

,Stock_Code,Date,Mean_Sentiment_Score
0,ADANIPORTS,"Aug 09, 2022",0.0
